In [14]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [15]:
data_folder = "/Users/aviadsusman/PycharmProjects/Sinai_FCNN/data"

data_frames = []

for file_name in os.listdir(data_folder):
    if not (file_name.startswith('labels') or file_name.startswith('data')):
        file_path = os.path.join(data_folder, file_name)
        data_frame = pd.read_csv(file_path)
        data_frames.append(data_frame)

# Concatenate all data frames into one
X = pd.concat(data_frames, axis=1).to_numpy()
X = np.asarray(X).astype(np.float32)
y = pd.read_csv('/Users/aviadsusman/PycharmProjects/Sinai_FCNN/data/labels.csv', header=None).to_numpy()

In [16]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
def train_model(X_train, y_train, num_hidden_layers):
    model = tf.keras.Sequential()

    # Input layer
    model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))

    # Hidden layers
    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))

    # Output layer
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    return model

In [18]:
num_hidden_layers_list = [3, 4, 5]

In [19]:
for num_hidden_layers in num_hidden_layers_list:
    print(f"Training with {num_hidden_layers} hidden layers:")

    accuracies = []
    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        model = train_model(X_train_fold, y_train_fold, num_hidden_layers)
        y_val_pred = model.predict(X_val_fold)
        y_val_pred = (y_val_pred >= 0.5).astype(int)

        accuracy = accuracy_score(y_val_fold, y_val_pred)
        accuracies.append(accuracy)

    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    print(f"Mean accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}\n")

Training with 3 hidden layers:
95/95 [==============================] - 0s 2ms/step
Mean accuracy: 0.7861 ± 0.1586

Training with 4 hidden layers:
95/95 [==============================] - 0s 3ms/step
Mean accuracy: 0.8672 ± 0.0053

Training with 5 hidden layers:
95/95 [==============================] - 0s 2ms/step
Mean accuracy: 0.8672 ± 0.0050
